In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imb-trn-cifar10/trn_lab_imb/trn_lab_imb
/kaggle/input/imb-trn-cifar10/BTestset/trn_lab_b
/kaggle/input/imb-trn-cifar10/BTestset/trn_img_b
/kaggle/input/imb-trn-cifar10/AutoEncoder_v2/bst_dec.pth
/kaggle/input/imb-trn-cifar10/AutoEncoder_v2/bst_enc.pth
/kaggle/input/imb-trn-cifar10/SMOTEd_v2/trn_lab
/kaggle/input/imb-trn-cifar10/SMOTEd_v2/trn_img
/kaggle/input/imb-trn-cifar10/NewKmeanAE/NewKmeanAE/0_bst_enc.pth
/kaggle/input/imb-trn-cifar10/NewKmeanAE/NewKmeanAE/0_bst_dec.pth
/kaggle/input/imb-trn-cifar10/ImbTestset/trn_lab_imb
/kaggle/input/imb-trn-cifar10/ImbTestset/trn_img_imb
/kaggle/input/imb-trn-cifar10/SMOTEd_Original/trn_lab
/kaggle/input/imb-trn-cifar10/SMOTEd_Original/trn_img
/kaggle/input/imb-trn-cifar10/trn_img_imb/trn_img_imb
/kaggle/input/imb-trn-cifar10/AutoDecoder_Original/0_bst_enc.pth
/kaggle/input/imb-trn-cifar10/AutoDecoder_Original/0_bst_dec.pth


In [2]:
# -*- coding: utf-8 -*-

import collections
import torch
import torch.nn as nn
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os
print(torch.version.cuda) #10.1
import time
t0 = time.time()
##############################################################################
"""args for models"""

args = {}
args['dim_h'] = 64          # factor controlling size of hidden layers
args['n_channel'] = 3       # number of channels in the input data 

args['n_z'] = 600 #600     # number of dimensions in latent space. 

args['sigma'] = 1.0        # variance in n_z
args['lambda'] = 0.01      # hyper param for weight of discriminator loss
args['lr'] = 0.0002        # learning rate for Adam optimizer .000
args['epochs'] = 500 #50         # how many epochs to run for
args['batch_size'] = 100   # batch size for SGD
args['save'] = True        # save weights at each epoch of training if True
args['train'] = True       # train networks if True, else load networks from

args['dataset'] = 'cifar10' #'fmnist' # specify which dataset to use

##############################################################################

## create encoder model and decoder model
class Encoder(nn.Module):
    def __init__(self, args):
        super(Encoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        
        # convolutional filters, work excellent with image data
        self.conv = nn.Sequential(
            nn.Conv2d(self.n_channel, self.dim_h, 4, 2, 1, bias=False),
            #nn.ReLU(True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(self.dim_h, self.dim_h * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 2),
            #nn.ReLU(True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(self.dim_h * 2, self.dim_h * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 4),
            #nn.ReLU(True),
            nn.LeakyReLU(0.2, inplace=True),
            
            
            #nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 2, 1, bias=False),
            
            #3d and 32 by 32
            nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 1, 0, bias=False),
            
            nn.BatchNorm2d(self.dim_h * 8), # 40 X 8 = 320
            #nn.ReLU(True),
            nn.LeakyReLU(0.2, inplace=True) )#,
            #nn.Conv2d(self.dim_h * 8, 1, 2, 1, 0, bias=False))
            #nn.Conv2d(self.dim_h * 8, 1, 4, 1, 0, bias=False))
        # final layer is fully connected
        self.fc = nn.Linear(self.dim_h * (2 ** 3), self.n_z)
        

    def forward(self, x):
        #print('enc')
        #print('input ',x.size()) #torch.Size([100, 3,32,32])
        x = self.conv(x)
        #print('aft conv ',x.size()) #torch.Size([100, 320, 2, 2]) with 
        #nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 2, 1, bias=False),
        #vs torch.Size([128, 320, 1, 1])
        #aft conv  torch.Size([100, 320, 1, 1]) with 
        #nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 1, 0, bias=False),
        x = x.squeeze()
        #print('aft squeeze ',x.size()) #torch.Size([128, 320])
        #aft squeeze  torch.Size([100, 320])
        x = self.fc(x)
        #print('out ',x.size()) #torch.Size([128, 20])
        #out  torch.Size([100, 300])
        return x


class Decoder(nn.Module):
    def __init__(self, args):
        super(Decoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']

        # first layer is fully connected
        self.fc = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 8 * 9 * 9),
            nn.ReLU())

        # deconvolutional filters, essentially inverse of convolutional filters
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(self.dim_h * 8, self.dim_h * 4, 4),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 4, self.dim_h * 2, 4),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 2, 3, 4, stride=2),
            #nn.Sigmoid()
            nn.Tanh())

    def forward(self, x):
        #print('dec')
        #print('input ',x.size())
        x = self.fc(x)
        x = x.view(-1, self.dim_h * 8, 9, 9)
        x = self.deconv(x)
        return x

##############################################################################

def biased_get_class1(c):
    
    xbeg = dec_x[dec_y == c]
    ybeg = dec_y[dec_y == c]
    
    return xbeg, ybeg
    #return xclass, yclass


def G_SM1(X, y,n_to_sample,cl):

    
    # fitting the model
    n_neigh = 5 + 1
    nn = NearestNeighbors(n_neighbors=n_neigh, n_jobs=1)
    nn.fit(X)
    dist, ind = nn.kneighbors(X)

    # generating samples
    base_indices = np.random.choice(list(range(len(X))),n_to_sample)
    neighbor_indices = np.random.choice(list(range(1, n_neigh)),n_to_sample)

    X_base = X[base_indices]
    X_neighbor = X[ind[base_indices, neighbor_indices]]

    samples = X_base + np.multiply(np.random.rand(n_to_sample,1),
            X_neighbor - X_base)

    #use 10 as label because 0 to 9 real classes and 1 fake/smoted = 10
    return samples, [cl]*n_to_sample

#############################################################################
np.printoptions(precision=5,suppress=True)

dtrnimg = '/kaggle/input/imb-trn-cifar10/trn_img_imb'
dtrnlab = '/kaggle/input/imb-trn-cifar10/trn_lab_imb'

ids = os.listdir(dtrnimg)
idtri_f = [os.path.join(dtrnimg, image_id) for image_id in ids]
idtri_f.sort()
print(idtri_f)

ids = os.listdir(dtrnlab)
idtrl_f = [os.path.join(dtrnlab, image_id) for image_id in ids]
idtrl_f.sort()
print(idtrl_f)

#path on the computer where the models are stored
modpth = '/kaggle/input/imb-trn-cifar10/NewKmeanAE/NewKmeanAE'

encf = []
decf = []
#for p in range(5):
for p in range(1):
    enc = modpth + '/' + str(p) + '_bst_enc.pth'
    #enc = '/kaggle/input/imb-trn-cifar10/AutoEncoder_v2/bst_enc.pth'
    dec = modpth + '/' + str(p) + '_bst_dec.pth'
    #dec = '/kaggle/input/imb-trn-cifar10/AutoEncoder_v2/bst_dec.pth'
    encf.append(enc)
    decf.append(dec)
    print(enc)
    print(dec)
    print()

#for m in range(5):
for m in range(1):
    print(m)
    trnimgfile = idtri_f[m]
    trnlabfile = idtrl_f[m]
    print(trnimgfile)
    print(trnlabfile)
    dec_x = np.loadtxt(trnimgfile) 
    dec_y = np.loadtxt(trnlabfile)

    print('train imgs before reshape ',dec_x.shape) #(44993, 3072) 45500, 3072)
    print('train labels ',dec_y.shape) #(44993,) (45500,)

    dec_x = dec_x.reshape(dec_x.shape[0],3,32,32)

    print('decy ',dec_y.shape)
    print(collections.Counter(dec_y))
    
    print('train imgs after reshape ',dec_x.shape) #(45000,3,32,32)

    classes = ('0', '1', '2', '3', '4',
           '5', '6', '7', '8', '9')
    
    #generate some images 
    train_on_gpu = torch.cuda.is_available()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    path_enc = encf[m]
    path_dec = decf[m]

    encoder = Encoder(args)
    encoder.load_state_dict(torch.load(path_enc), strict=False)
    encoder = encoder.to(device)

    decoder = Decoder(args)
    decoder.load_state_dict(torch.load(path_dec), strict=False)
    decoder = decoder.to(device)

    encoder.eval()
    decoder.eval()

    imbal = [4500, 2000, 1000, 800, 600, 500, 400, 250, 150, 80]
    #imbal = [4000, 2000, 1000, 750, 500, 350, 200, 100, 60, 40]

    resx = []
    resy = []

    for i in range(1,10):
        xclass, yclass = biased_get_class1(i)
        print(xclass.shape) #(500, 3, 32, 32)
        print(yclass[0]) #(500,)
            
        #encode xclass to feature space
        xclass = torch.Tensor(xclass)
        xclass = xclass.to(device)
        xclass = encoder(xclass)
        print(xclass.shape) #torch.Size([500, 600])
            
        xclass = xclass.detach().cpu().numpy()
        n = imbal[0] - imbal[i]
        xsamp, ysamp = G_SM1(xclass,yclass,n,i)
        print(xsamp.shape) #(4500, 600)
        print(len(ysamp)) #4500
        ysamp = np.array(ysamp)
        print(ysamp.shape) #4500   
    
        """to generate samples for resnet"""   
        xsamp = torch.Tensor(xsamp)
        xsamp = xsamp.to(device)
        #xsamp = xsamp.view(xsamp.size()[0], xsamp.size()[1], 1, 1)
        #print(xsamp.size()) #torch.Size([10, 600, 1, 1])
        ximg = decoder(xsamp)

        ximn = ximg.detach().cpu().numpy()
        print(ximn.shape) #(4500, 3, 32, 32)
        #ximn = np.expand_dims(ximn,axis=1)
        print(ximn.shape) #(4500, 3, 32, 32)
        resx.append(ximn)
        resy.append(ysamp)
        #print('resx ',resx.shape)
        #print('resy ',resy.shape)
        #print()
    
    resx1 = np.vstack(resx)
    resy1 = np.hstack(resy)
    #print(resx1.shape) #(34720, 3, 32, 32)
    #resx1 = np.squeeze(resx1)
    print(resx1.shape) #(34720, 3, 32, 32)
    print(resy1.shape) #(34720,)

    resx1 = resx1.reshape(resx1.shape[0],-1)
    print(resx1.shape) #(34720, 3072)
    
    dec_x1 = dec_x.reshape(dec_x.shape[0],-1)
    print('decx1 ',dec_x1.shape)
    combx = np.vstack((resx1,dec_x1))
    comby = np.hstack((resy1,dec_y))

    print(combx.shape) #(45000, 3, 32, 32)
    print(comby.shape) #(45000,)

    #ifile = '.../MNIST/trn_img_f/' + \
        #str(m) + '_trn_img.txt'
    ifile = '/kaggle/working/' + str(m) +'_trn_img.gz'
    np.savetxt(ifile, combx)
    
    #lfile = '.../MNIST/trn_lab_f/' + \
        #str(m) + '_trn_lab.txt'
    lfile = '/kaggle/working/' + str(m) + '_trn_lab.gz'
    np.savetxt(lfile,comby) 
    print()

t1 = time.time()
print('final time(min): {:.2f}'.format((t1 - t0)/60))

11.0
['/kaggle/input/imb-trn-cifar10/trn_img_imb/trn_img_imb']
['/kaggle/input/imb-trn-cifar10/trn_lab_imb/trn_lab_imb']
/kaggle/input/imb-trn-cifar10/NewKmeanAE/NewKmeanAE/0_bst_enc.pth
/kaggle/input/imb-trn-cifar10/NewKmeanAE/NewKmeanAE/0_bst_dec.pth

0
/kaggle/input/imb-trn-cifar10/trn_img_imb/trn_img_imb
/kaggle/input/imb-trn-cifar10/trn_lab_imb/trn_lab_imb
train imgs before reshape  (10280, 3072)
train labels  (10280,)
decy  (10280,)
Counter({0.0: 4500, 1.0: 2000, 2.0: 1000, 3.0: 800, 4.0: 600, 5.0: 500, 6.0: 400, 7.0: 250, 8.0: 150, 9.0: 80})
train imgs after reshape  (10280, 3, 32, 32)
(2000, 3, 32, 32)
1.0
torch.Size([2000, 600])
(2500, 600)
2500
(2500,)
(2500, 3, 32, 32)
(2500, 3, 32, 32)
(1000, 3, 32, 32)
2.0
torch.Size([1000, 600])
(3500, 600)
3500
(3500,)
(3500, 3, 32, 32)
(3500, 3, 32, 32)
(800, 3, 32, 32)
3.0
torch.Size([800, 600])
(3700, 600)
3700
(3700,)
(3700, 3, 32, 32)
(3700, 3, 32, 32)
(600, 3, 32, 32)
4.0
torch.Size([600, 600])
(3900, 600)
3900
(3900,)
(3900, 3, 32

<a href="/kaggle/working/trn_img.gz"> Download File </a>